In [1]:
import os

In [2]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example'

In [ ]:
# now we dont need to update config.yaml here
# for model evaluation here, no need
# directly define the entity - path etc,

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    #mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [29]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chickenfecalimages",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [30]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [31]:
# import the model
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [32]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        "function to load the model"
        return tf.keras.models.load_model(path)
    

    def evaluation(self): 
        "performing evaluation"  
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        "saving in scores variable"
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        # saving the scores output in json file
        save_json(path=Path("scores.json"), data=scores)

In [33]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    

except Exception as e:
   raise e

# in the left pane below, you can find scores.json file
# and then convert this notebook into modular piece.

[2025-01-02 12:31:04,964: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 12:31:04,967: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 12:31:04,970: INFO: common: created directory at: artifacts]
Found 14 images belonging to 2 classes.
1/1 [==============================] - 3s 3s/step - loss: 27.6701 - accuracy: 0.5000
[2025-01-02 12:31:08,780: INFO: common: json file saved at: scores.json]
